In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-05-25T09:29:08.687217-04:00

Python implementation: CPython
Python version       : 3.7.11
IPython version      : 7.10.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 4.15.0-161-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit



In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging

In [4]:
%watermark --iversions

pandas : 1.3.5
numpy  : 1.19.2
optuna : 2.10.1
dask   : 2022.1.0
logging: 0.5.1.2
xgboost: 1.6.2



In [5]:
!nvidia-smi -L

GPU 0: Tesla V100-DGXS-32GB (UUID: GPU-d752af09-1f62-bf3b-4f70-78b84e9e41f6)
GPU 1: Tesla V100-DGXS-32GB (UUID: GPU-054a4a35-f98a-3ebc-9100-0f697c246b43)
GPU 2: Tesla V100-DGXS-32GB (UUID: GPU-454525d4-bebd-7fb8-0ba3-3b85e2f99abd)
GPU 3: Tesla V100-DGXS-32GB (UUID: GPU-af0fb74e-f5eb-0833-17ff-494cc6bdcee1)


In [6]:
cluster = LocalCUDACluster(n_workers=4)
client = Client(cluster)

In [7]:
!nvidia-smi

Thu May 25 09:29:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   40C    P0    51W / 300W |    667MiB / 32499MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [8]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}_l.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}_l.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [9]:
train = pd.read_csv('../input/train.csv.zip')

shift = 200

target0 = train['loss'].values
target = np.log(target0+shift)

In [10]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective': 'reg:squarederror', 
        'tree_method':'gpu_hist',  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['rmse']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = np.exp(predictions) - shift
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    mae = mean_absolute_error(target0, train_oof)
    
    return mae

In [11]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_l.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_allstate_l.db", study_name="five_fold_optuna_xgb_l", direction='minimize')

In [12]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 46% CPU time recently (threshold: 10%)


CPU times: user 35.1 s, sys: 6.94 s, total: 42 s
Wall time: 6min 21s


In [13]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1168.486142,3.583454,0.938987,rmse,0.004402,0.005453,14,135,0.800484,COMPLETE
1,1,1142.704954,0.142099,0.369230,rmse,8.267987,0.025918,18,75,0.714054,COMPLETE
2,2,1171.449333,0.042546,0.309648,rmse,0.278567,0.015465,4,209,0.524717,COMPLETE


In [14]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_l.csv', index=False)
df

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 77% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 77% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 77% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 77% CPU time recently (thre

CPU times: user 2min 42s, sys: 40.1 s, total: 3min 22s
Wall time: 35min 45s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1168.486142,3.583454,0.938987,rmse,0.004402,0.005453,14,135,0.800484,COMPLETE
1,1,1142.704954,0.142099,0.369230,rmse,8.267987,0.025918,18,75,0.714054,COMPLETE
2,2,1171.449333,0.042546,0.309648,rmse,0.278567,0.015465,4,209,0.524717,COMPLETE
3,3,2869.666235,0.002074,0.483770,rmse,4.948715,0.001323,16,152,0.997211,COMPLETE
4,4,1141.258043,0.051828,0.976007,rmse,0.334002,0.009005,24,113,0.980396,COMPLETE
5,5,1145.218284,2.088871,0.355785,rmse,0.011890,0.044316,24,299,0.913838,COMPLETE
6,6,1158.671766,0.014340,0.373486,rmse,0.090945,0.006021,22,238,0.562549,COMPLETE
7,7,1177.735612,8.483505,0.820162,rmse,0.024255,0.007864,6,203,0.447976,COMPLETE


In [15]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_l.csv', index=False)
df.head()

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (thre

CPU times: user 44min 16s, sys: 9min 25s, total: 53min 42s
Wall time: 8h 22min 17s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1168.486142,3.583454,0.938987,rmse,0.004402,0.005453,14,135,0.800484,COMPLETE
1,1,1142.704954,0.142099,0.369230,rmse,8.267987,0.025918,18,75,0.714054,COMPLETE
2,2,1171.449333,0.042546,0.309648,rmse,0.278567,0.015465,4,209,0.524717,COMPLETE
3,3,2869.666235,0.002074,0.483770,rmse,4.948715,0.001323,16,152,0.997211,COMPLETE
4,4,1141.258043,0.051828,0.976007,rmse,0.334002,0.009005,24,113,0.980396,COMPLETE


In [16]:
df

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1168.486142,3.583454,0.938987,rmse,0.004402,0.005453,14,135,0.800484,COMPLETE
1,1,1142.704954,0.142099,0.369230,rmse,8.267987,0.025918,18,75,0.714054,COMPLETE
2,2,1171.449333,0.042546,0.309648,rmse,0.278567,0.015465,4,209,0.524717,COMPLETE
3,3,2869.666235,0.002074,0.483770,rmse,4.948715,0.001323,16,152,0.997211,COMPLETE
4,4,1141.258043,0.051828,0.976007,rmse,0.334002,0.009005,24,113,0.980396,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...
103,103,1135.843194,1.236278,0.541091,rmse,0.088771,0.015721,21,222,0.946697,COMPLETE
104,104,1136.043002,0.883243,0.542647,rmse,0.084925,0.017914,21,222,0.949701,COMPLETE
105,105,1139.168866,1.104670,0.537168,rmse,0.086147,0.009173,21,222,0.949910,COMPLETE
106,106,1135.595662,2.119515,0.440832,rmse,0.041819,0.014413,22,203,0.957350,COMPLETE


In [17]:
df.value.min()

1135.5412451156963